# Hyperparameter Optimization with Optuna

Welcome to the Optuna-based hyperparameter optimization tutorial! In this interactive notebook, you will explore world of hyperparameter tuning for a Convolutional Neural Network (CNN) specifically aimed at image classification using the CIFAR-10 dataset. Hyperparameter optimization is pivotal in enhancing model performance, making your models more accurate and efficient.

Optuna, a robust and versatile library, plays a central role in automating and streamlining this process. It empowers you to navigate through complex hyperparameter spaces with ease. In this tutorial, you will engage with Optuna's core functionalities, and you'll also have the opportunity to construct a flexible CNN architecture. This adaptable design is essential for understanding how models can be fine-tuned effortlessly to suit various hyperparameter configurations.

Throughout this session, you will:
- Learn how to set up and execute an Optuna study, incorporating all essential elements required for effective hyperparameter optimization.
- Perform a thorough analysis of the results to evaluate how different hyperparameters influence model performance, gaining insights into their practical impact.

Additionally, this tutorial includes an optional section where you will compare two prevalent methods of hyperparameter optimization: Optuna's default sampling method (Tree-structured Parzen Estimator, or TPE) and the traditional Grid Search method. This comparison will not only highlight the strengths of Optuna but also provide a clearer perspective on how it can outperform conventional optimization techniques.


In [3]:
import torch
import torch.nn as nn 
import torch.optim as optim
import optuna
import matplotlib.pyplot as plt
import torch.nn.functional as F
from pprint import pprint
import helper

helper.set_seed(15)


/Users/amitvermaknw/Desktop/app/ML/deep-learning-with-pytorch/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(device)

mps


## Hyperparameter Optimization for CNNs on CIFAR-10

In this section, you explore the vital task of finding the optimal hyperparameters for a Convolutional Neural Network (CNN) tailored to the CIFAR-10 dataset. 
Utilizing Optuna, a sophisticated framework for hyperparameter optimization, your goal is to streamline and automate the process, ensuring efficiency and effectiveness. 
The selection of hyperparameters is notably intensive computationally and depends on various factors including the architecture of the model, the dataset characteristics, and the specific training processes involved. These elements, collectively and individually, have significant impacts on the performance outcomes of the model.

### Defining a Flexible CNN Architecture

The model architecture here is deliberately designed to be flexible, accommodating variability in its layers which is pivotal for adapting to different hyperparameter configurations suggested by Optuna during optimization trials.  
The architecture is defined in a modular manner, allowing for easy adjustments and experimentation with different layer configurations, activation functions, and other hyperparameters. 

`FlexibleCNN` is a class that encapsulates the architecture of the CNN model:

* **`__init__`**: The constructor initializes the model's feature extraction layers.
>    * It constructs a series of convolutional blocks based on the `n_layers` parameter. Each block is a sequence of `nn.Conv2d`, `nn.ReLU`, and `nn.MaxPool2d`.
>    * The `in_channels` for each block is set to the `out_channels` of the preceding block to ensure a seamless data flow.
>    * All blocks are combined into a single `nn.Sequential` module assigned to the `.features` attribute, which handles feature extraction.
>    * The classifier, `.classifier`, is initially set to `None` and will be constructed dynamically later.
 * **`_create_classifier`**: This helper method dynamically builds the classifier part of the network.
>    * It's called during the first forward pass once the input size for the linear layers is known.
 * **`forward`**: This method defines the forward pass of the model.
>    * The input `x` first passes through the `.features` layers.
>    * The output from the feature extractor is flattened to determine the input size for the classifier.
>    * If the `.classifier` has not been created yet, it calls `_create_classifier` to build it on the fly.
>    * Finally, the flattened data is passed through the `.classifier` to produce the final output.

In [5]:
class FlexibleCNN(nn.Module):
    """
    A flexible Convolutional Neural Network with a dynamically created classifier.

    This CNN's architecture is defined by the provided hyperparameters,
    allowing for a variable number of convolutional layers. The classifier
    (fully connected layers) is constructed during the first forward pass
    to adapt to the output size of the convolutional feature extractor.
    """
    def __init__(self, n_layers, n_filters, kernel_sizes, dropout_rate, fc_size):
        """
        Initializes the feature extraction part of the CNN.

        Args:
            n_layers: The number of convolutional blocks to create.
            n_filters: A list of integers specifying the number of output
                       filters for each convolutional block.
            kernel_sizes: A list of integers specifying the kernel size for
                          each convolutional layer.
            dropout_rate: The dropout probability to be used in the classifier.
            fc_size: The number of neurons in the hidden fully connected layer.
        """
        super(FlexibleCNN, self).__init__()

        #Initialize an empty list to hold the convolution blocks
        blocks = []
        #Set the initial number of input channels for RGB images
        in_channels = 3

        #Loop to construct each convolutional block
        for i in range(n_layers):
            #Get the parameters for current convolution layer
            out_channels = n_filters[i]
            kernel_size = kernel_sizes[i]

            #Calculate padding to maintain the input spatial dimension('same' padding)
            padding = (kernel_size-1) // 2

            #Define a block as a sequence of conv, ReLU and MaxPool. layers
            block = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )

            #Add the newly created block to the list
            blocks.append(block)

            #Update the number of input channels for the next block
            in_channels = out_channels

        #Combine all blocks inot a single feature extractor module
        self.features = nn.Sequential(*blocks)

        #Store hyperparameters needed for building the classifier later
        self.dropout_rate = dropout_rate
        self.fc_size = fc_size

        #The classifier will be initialized dynamically in the forward pass
        self.classifier = None

    def _create_classifier(self, flattened_size, device):
        """
        Dynamically creates and initializes the classifier part of the network.

        This helper method is called during the first forward pass to build the
        fully connected layers based on the feature map size from the
        convolutional base.

        Args:
            flattened_size: The number of input features for the first linear
                            layer, determined from the flattened feature map.
            device: The device to which the new classifier layers should be moved.
        """

        #Define the classifier's architecture
        self.classifier = nn.Sequential(
            nn.Dropout(self.dropout_rate),
            nn.Linear(flattened_size, self.fc_size),
            nn.ReLU(inplace=True),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.fc_size, 100)
        ).to(device)

    def forward(self, x):
        """
        Defines the forward pass of the model.

        Args:
            x: The input tensor of shape (batch_size, channels, height, width).

        Returns:
            The output logits from the classifier.
        """
        #Get the device of the input tensor to ensure consistency
        device = x.device

        #Pass the input through the feature extraction layer
        x=self.features(x)

        #Flatten the feature map to prepare it for the fully connected layers
        flattened = torch.flatten(x, 1)
        flattened_size = flattened.size(1)

        #If the classifier has not been created yet, initialize it
        if self.classifier is None:
            self._create_classifier(flattened_size, device)

        #pass the flattened feature through the classifier to get the final output
        return self.classifier(flattened)

## Defining the Optuna Objective Function

The objective function is the core of the hyperparameter optimization process, being the function that Optuna will repeatedly call to evaluate different hyperparameter configurations.
This function encapsulates the entire training and evaluation process, including the definition of the CNN model architecture, the optimizer, the data loaders, the training loop, and the evaluation metrics.
Within this function, you define the search space for hyperparameters using `trial.suggest_*` methods, which allow Optuna to sample hyperparameters from a defined range or set of values. 
For a full list of available `suggest_*` methods, you can refer to the [Optuna documentation](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html).

*The objective function is designed to return a single scalar value, which represents the performance of the model for the given hyperparameters*. 
In your case, the aim to maximize the accuracy of the model on the validation set, which is computed using the `evaluate_accuracy` function.

**Dynamic Layer Initialization**: A noteworthy addition to this objective function is the initialization step involving a dummy input. Because the `FlexibleCNN` creates its classifier layers dynamically during the first forward pass, these parameters do not exist immediately after the model is instantiated.
- **Why use a dummy input?** Passing data through the model forces it to calculate the flattened feature size and build the classifier layers. You must do this *before* defining the optimizer so that `model.parameters()` includes the classifier weights. Otherwise, the optimizer would only track the feature extractor, leaving the classifier untrained.
>
- **Why these dimensions?** The tensor `torch.randn(1, 3, 32, 32)` is used to mimic the structure of the CIFAR-10 dataset. It represents a single image (batch size of 1) with 3 color channels (RGB) and a resolution of `32x32` pixels.

Observe that some hyperparameters are defined as fixed values, such as the number of epochs, the batch size, and the learning rate.

In [8]:
def objective(trial, device):
    """
    Defines the objective function for hyperparameter optimization using Optuna.

    For each trial, this function samples a set of hyperparameters,
    constructs a model, trains it for a fixed number of epochs, evaluates
    its performance on a validation set, and returns the accuracy. Optuna
    uses the returned accuracy to guide its search for the best
    hyperparameter combination.

    Args:
        trial: An Optuna `Trial` object, used to sample hyperparameters.
        device: The device ('cpu' or 'cuda') for model training and evaluation.

    Returns:
        The validation accuracy of the trained model as a float.
    """

    #Sample hyperparameters for the feature extractor using the Optuna trial
    n_layers = trial.suggest_int("n_layer", 1, 3)
    n_filters=[trial.suggest_int(f"n_filter_{i}", 16, 128) for i in range(n_layers)]
    kernal_sizes = [trial.suggest_categorical(f"kernel_size_{i}", [3,5]) for i in range(n_layers)]

    #Sample hyperparameters for the classifier
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    fc_size = trial.suggest_int("fc_size", 64, 256)

    #Instantiate the model with the sampled hyperparameters
    model = FlexibleCNN(n_layers, n_filters, kernal_sizes, dropout_rate, fc_size).to(device)

    #Initialize the dynamic classifier layer bt passing a dummy input through the model
    #This ensure all parameters are instantiated before the optimizer is defined
    dummy_input = torch.rand(1,3,32, 32).to(device)
    model(dummy_input)

    #Define fixed training parameters: lr, loss function and optimizer
    learning_rate = 0.001
    loss_fcn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)

    #Define fixed daata loading parameters and crete data loader
    batch_size=128
    train_loader, val_loader = helper.get_dataset_dataloaders(batch_size=batch_size)

    #Define the fixed number of epochs for training
    n_epochs = 10

    #Train model using a helper function
    helper.train_model(
        model=model,
        optimizer=optimizer,
        loss_fcn=loss_fcn,
        train_dataloader=train_loader,
        n_epochs=n_epochs,
        device=device
    )

    #Evaluate the trained model's accuracy on the validaation set
    accuracy = helper.evaluate_accuracy(model, val_loader, device)

    return accuracy

## Running the Optuna Study

Once that the objective function is defined, an Optuna study is created to manage the hyperparameter optimization process.
The study is responsible for running the objective function multiple times with different hyperparameter configurations, allowing Optuna to explore the search space and find the best hyperparameters.
In this case, your goal is to **maximize the accuracy** of the CNN model on the CIFAR-10 dataset, this is why we use `direction='maximize'` when creating the study.
The `optimize` method of the study is called to start the optimization process, which will run the objective function for a defined number of trials.

A lambda function is used to pass the device to the objective function, allowing the model to be trained on the specified device
*Note*: you can also pass other parameters to the objective function using the lambda function, if needed.

**NOTE:** the code below will take about 8 minutes to run.

In [9]:
#Create a study object and optimie the objective function
study = optuna.create_study(direction='maximize') #The goal in this case is to maximize accuracy

#Start the optimization process 
n_trials = 20
study.optimize(lambda trial: objective(trial, device), n_trials=n_trials)

[I 2026-01-30 11:06:46,717] A new study created in memory with name: no-name-be59388d-7752-4469-aab3-f2b03f6c4c6f
/Users/amitvermaknw/Desktop/app/ML/deep-learning-with-pytorch/.venv/lib/python3.14/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")
Training - Current Epoch: 6:  60%|██████    | 6/10 [00:07<00:05,  1.37s/it] 

Epoch 5 - Train Loss: 1.0618


Training - Current Epoch: 10: 100%|██████████| 10/10 [00:14<00:00,  1.49s/it]


Epoch 10 - Train Loss: 0.6576
Training complete!



[I 2026-01-30 11:07:03,158] Trial 0 finished with value: 0.5745 and parameters: {'n_layer': 1, 'n_filter_0': 120, 'kernel_size_0': 3, 'dropout_rate': 0.25596530447594396, 'fc_size': 187}. Best is trial 0 with value: 0.5745.


...

Epoch 5 - Train Loss: 1.4231


Training - Current Epoch: 10: 100%|██████████| 10/10 [00:14<00:00,  1.49s/it]


Epoch 10 - Train Loss: 1.1666
Training complete!



[I 2026-01-30 11:13:58,399] Trial 19 finished with value: 0.563 and parameters: {'n_layer': 2, 'n_filter_0': 40, 'n_filter_1': 23, 'kernel_size_0': 5, 'kernel_size_1': 5, 'dropout_rate': 0.411496977086487, 'fc_size': 252}. Best is trial 12 with value: 0.6255.


Evaluation complete.


### Analyzing the Results
After the optimization process is complete, you can analyze the results to understand which hyperparameters yielded the best performance.
The `study` object contains a wealth of information about the trials, including the hyperparameters sampled, the corresponding performance metrics, and the best trial.

You can access the full DataFrame of trials using the `trials_dataframe()` method, which provides a comprehensive overview of all the trials conducted during the optimization process.
This DataFrame includes columns for the trial number, hyperparameters, and the objective value (in our case, the accuracy).

To access the best hyperparameters and the best trial, you can use the `best_trial` attributes of the study object.

**Note:** these results may change every time you re-run the training study.

In [10]:
#Extract the dataframe with the result 
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_dropout_rate,params_fc_size,params_kernel_size_0,params_kernel_size_1,params_kernel_size_2,params_n_filter_0,params_n_filter_1,params_n_filter_2,params_n_layer,state
0,0,0.5745,2026-01-30 11:06:46.718465,2026-01-30 11:07:03.158754,0 days 00:00:16.440289,0.255965,187,3,NaN,NaN,120,NaN,NaN,1,COMPLETE
1,1,0.4955,2026-01-30 11:07:03.159221,2026-01-30 11:07:15.534493,0 days 00:00:12.375272,0.296383,74,3,3.0,5.0,20,36.0,37.0,3,COMPLETE
2,2,0.5700,2026-01-30 11:07:15.534888,2026-01-30 11:07:31.201139,0 days 00:00:15.666251,0.446501,255,5,5.0,NaN,54,16.0,NaN,2,COMPLETE
3,3,0.5800,2026-01-30 11:07:31.201581,2026-01-30 11:07:47.829249,0 days 00:00:16.627668,0.268576,188,3,3.0,5.0,56,55.0,82.0,3,COMPLETE
4,4,0.5890,2026-01-30 11:07:47.829597,2026-01-30 11:08:03.606687,0 days 00:00:15.777090,0.263554,165,5,NaN,NaN,89,NaN,NaN,1,COMPLETE
5,5,0.5950,2026-01-30 11:08:03.607125,2026-01-30 11:08:37.363217,0 days 00:00:33.756092,0.247971,154,5,5.0,3.0,118,84.0,86.0,3,COMPLETE
6,6,0.5560,2026-01-30 11:08:37.363562,2026-01-30 11:08:53.436992,0 days 00:00:16.073430,0.446870,170,3,3.0,3.0,125,42.0,95.0,3,COMPLETE
7,7,0.6160,2026-01-30 11:08:53.437292,2026-01-30 11:09:21.238821,0 days 00:00:27.801529,0.230798,219,3,5.0,NaN,97,94.0,NaN,2,COMPLETE
8,8,0.6210,2026-01-30 11:09:21.239173,2026-01-30 11:09:40.799672,0 days 00:00:19.560499,0.248242,232,5,5.0,NaN,84,35.0,NaN,2,COMPLETE
9,9,0.5655,2026-01-30 11:09:40.799951,2026-01-30 11:09:56.628444,0 days 00:00:15.828493,0.211654,171,3,3.0,NaN,116,42.0,NaN,2,COMPLETE


In [12]:
#Extract and print the best trial
best_trial = study.best_trial
print("Best Trial:")
print(f"Value Accuracy: {best_trial.value:.4f}")

print("Hyperpaameters:")
print(best_trial.params)

Best Trial:
Value Accuracy: 0.6255
Hyperpaameters:
{'n_layer': 2, 'n_filter_0': 90, 'n_filter_1': 123, 'kernel_size_0': 5, 'kernel_size_1': 5, 'dropout_rate': 0.12530665372694055, 'fc_size': 251}


## Visualizing the Results

Optuna provides several built-in visualization functions to help analyze the results of the hyperparameter optimization process.
These visualizations can provide valuable insights into the optimization process and the impact of different hyperparameters on the model's performance:
- `plot_optimization_history`: This plot shows the optimization history of the objective function, allowing you to see how the performance of the model improved over time. It provides a visual representation of the objective values (in this case, accuracy) across different trials.
 
- `plot_param_importances`: This plot shows the importance of each hyperparameter in the optimization process. It helps identify which hyperparameters had the most significant impact on the model's performance, allowing you to focus on the most influential hyperparameters in future experiments.

- `plot_parallel_coordinate`: This plot visualizes the relationship between different hyperparameters and the objective function. It allows you to see how different hyperparameter configurations affected the model's performance, providing insights into the interactions between hyperparameters and their impact on the objective value.

In [ ]:
#Plotting the optimization history
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.title("Optimization History")
plt.show()

#Importance of hyperparameters
optuna.visualization.matplotlib.plot_param_importances(study)
plt.show()

ax = optuna.visualization.matplotlib.plot_parallel_coordinate(
    study, params=['n_layers', 'n_filters_0', 'kernel_size_0', 'dropout_rate', 'fc_size']
)

fig = ax.figure
fig.set_size_inches(12, 6, forward=True)
fig.tight_layout()